# Final Project

## Notes

In [1]:
'''
Using the following machine learning models for making predictions for the following portfolio optimization models:
- Mean-variance optimization (chapter 7 of Bodie, Investment book)
- Index model (chapter 8 of Bodie, Investment book)
- Capital asset pricing model (chapter 9 of Bodie, Investment book)
- Arbitrage pricing theory and multifactor model (chapter 10 of Bodie,
Investment book)
- Equity valuation model (chapter 18 of Bodie, Investment book)
- Black Litterman model (chapter 24 of Bodie, Investment book)
- Algorithmic trading (this could be restricted to the last month)
'''

'\nUsing the following machine learning models for making predictions for the following portfolio optimization models:\n- Mean-variance optimization (chapter 7 of Bodie, Investment book)\n- Index model (chapter 8 of Bodie, Investment book)\n- Capital asset pricing model (chapter 9 of Bodie, Investment book)\n- Arbitrage pricing theory and multifactor model (chapter 10 of Bodie,\nInvestment book)\n- Equity valuation model (chapter 18 of Bodie, Investment book)\n- Black Litterman model (chapter 24 of Bodie, Investment book)\n- Algorithmic trading (this could be restricted to the last month)\n'

## Load Data

In [2]:
import yfinance as yf
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from pypfopt import expected_returns, risk_models, EfficientFrontier
from sklearn.svm import SVR

tickers = ["AAPL", "AMZN", "BRK-B", "GOOGL", "META", "MSFT", "NVDA", "TSLA", "UNH", "XOM"]
portfolio_data = yf.download(tickers, period="5y")
portfolio_data = portfolio_data["Adj Close"]
portfolio_data.tail()

[*********************100%***********************]  10 of 10 completed


,AAPL,AMZN,BRK-B,GOOGL,META,MSFT,NVDA,TSLA,UNH,XOM
Date,,,,,,,,,,
2023-06-26,185.270004,127.330002,334.119995,118.339996,278.470001,328.600006,406.320007,241.050003,479.059998,104.290001
2023-06-27,188.059998,129.179993,335.339996,118.330002,287.049988,334.570007,418.760010,250.210007,482.559998,104.550003
2023-06-28,189.250000,129.039993,334.149994,120.180000,285.290009,335.850006,411.170013,256.239990,474.450012,105.400002
2023-06-29,189.589996,127.900002,336.910004,119.099998,281.529999,335.049988,408.220001,257.500000,476.440002,106.699997
2023-06-30,193.970001,130.360001,341.000000,119.699997,286.980011,340.540009,423.019989,261.769989,480.640015,107.250000


In [3]:
market_index_ticker = "^GSPC"  # S&P 500 index
market_index_data = yf.download(market_index_ticker, period="5y")
market_index_data = market_index_data["Adj Close"]
market_index_data.tail()

[*********************100%***********************]  1 of 1 completed


Date
2023-06-26    4328.819824
2023-06-27    4378.410156
2023-06-28    4376.859863
2023-06-29    4396.439941
2023-06-30    4450.379883
Name: Adj Close, dtype: float64

## Mean Variance Optimization

### Random Forest 

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    # Dependent variable - 10 consecutive days of stock prices
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    
    # Independent variable - stock price 10th day into the future
    dev_y = [data[i+10] for i in range(10,len(data)-10)]
    
    test = [data[i:i+10] for i in range(len(data)-20,len(data)-10)]
    
    reg = RandomForestRegressor()
    reg.fit(dev_x,dev_y)
    
    # Predict stock price for 10 future days
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)
# Construct covariance matrix of future stock prices
S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()

# Use capm to find expected returns on future prices
mu = expected_returns.capm_return(future_prices)
print(mu)

# Do mean variance optimization using efficient frontier
ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=0.02)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)


### SVM

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    dev_y = [data[i+10] for i in range(10, len(data)-10)]
    test = [data[i:i+10] for i in range(len(data)-20, len(data)-10)]
    
    reg = SVR()
    reg.fit(dev_x, dev_y)
    
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)

S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
mu = expected_returns.capm_return(future_prices)
print(mu)

ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)


ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)


### Neural Network

In [ ]:
future_prices = {}
for ticker in tickers:
    data = portfolio_data[ticker].tolist()
    dev_x = [data[i:i+10] for i in range(len(data)-20)]
    dev_y = [data[i+10] for i in range(10, len(data)-10)]
    test = [data[i:i+10] for i in range(len(data)-20, len(data)-10)]
    
    reg = MLPRegressor(hidden_layer_sizes=(100,100), random_state=42)  # You can adjust the hidden_layer_sizes and other parameters as needed
    reg.fit(dev_x, dev_y)
    
    pred = reg.predict(test)
    future_prices[ticker] = pred

future_prices = pd.DataFrame(future_prices)

S = risk_models.CovarianceShrinkage(future_prices).ledoit_wolf()
mu = expected_returns.capm_return(future_prices)
print(mu)

ef = EfficientFrontier(mu, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)



## Index model

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

class PortfolioOptimization:
    def __init__(self, market_index_data, portfolio_data, tickers):
        self.market_index_data = market_index_data
        self.portfolio_data = portfolio_data
        self.tickers = tickers
        
    def __init__(self, tickers, returns, market_future, reg_model):
        self.tickers = tickers
        self.returns = returns
        self.market_future = market_future
        self.reg_model = reg_model
        
        self.beta = {}
        self.alpha = {}
        self.residual_variance = {}
        self.weights = {}
        self.final_weight_market = None
        self.alpha_portfolio = None
        self.residual_variance_portfolio = None
        self.initial_position_portfolio = None
        self.beta_portfolio = None
        self.adjusted_position_portfolio = None
        self.risk_premium_portfolio = None
        self.portfolio_variance = None
        self.sharpe_ratio = None
        
    def calculate_returns(self):
        self.returns = {"MARKET": ((self.market_index_data / self.market_index_data.shift(1)) - 1).dropna().tolist()}
        for ticker in self.tickers:
            self.returns[ticker] = ((self.portfolio_data[ticker] / self.portfolio_data[ticker].shift(1)) - 1).dropna().tolist()

    def predict_market_returns(self):
        market_x = [self.returns["MARKET"][i:i+10] for i in range(len(self.returns["MARKET"])-20)]
        market_y = [self.returns["MARKET"][i+10] for i in range(10, len(self.returns["MARKET"])-10)]

        market_test = [self.returns["MARKET"][i:i+10] for i in range(len(self.returns["MARKET"])-20, len(self.returns["MARKET"])-10)]

        reg = SVR()
        reg.fit(market_x, market_y)
        
        market_future = reg.predict(market_test)
        return market_future

    def fit_single_index_model(self):
        for ticker in self.tickers:
            # ----
            stock_x = [returns[ticker][i:i+10] for i in range(len(returns[ticker])-20)]
            stock_y = [returns[ticker][i+10] for i in range(10, len(returns[ticker])-10)]

            stock_test = [returns[ticker][i:i+10]
                          for i in range(len(returns[ticker])-20, len(returns[ticker])-10)]

            reg = self.reg_model
            reg.fit(stock_x, stock_y)
            stock_future = reg.predict(stock_test)
            # ----

            single_index_reg = LinearRegression()
#             single_index_reg.fit(np.array(self.returns["MARKET"]).reshape(-1, 1), y=self.returns[ticker])
            single_index_reg.fit(np.array(market_future).reshape(-1, 1),y=stock_future)
            
#             stock_future = single_index_reg.predict(np.array(self.market_future).reshape(-1, 1))

            self.beta[ticker] = single_index_reg.coef_[0]
            self.alpha[ticker] = single_index_reg.intercept_

            y_pred = single_index_reg.predict(np.array(self.returns["MARKET"]).reshape(-1, 1))
            residuals = self.returns[ticker] - y_pred
            self.residual_variance[ticker] = np.var(residuals)

    def treynor_black_optimization(self, residual_variance_market=0.0114, risk_premium_market=0.056):
        # STEP 1:
        # Compute the initial position of each security:
        self.weights = {ticker: self.alpha[ticker] / self.residual_variance[ticker] for ticker in self.tickers}
#         print(f"Weights: {self.weights}\n")

        # STEP 2:
        # Scale the initial positions:
        total_weight = sum(self.weights.values())
        self.weights = {ticker: weight / total_weight for ticker, weight in self.weights.items()}
#         print(f"Scaled weights: {self.weights}\n")

        # STEP 3:
        # Compute the alpha of the active portfolio:
        self.alpha_portfolio = sum(self.weights[ticker] * self.alpha[ticker] for ticker in self.tickers)
#         print(f"Alpha of the active portfolio: {self.alpha_portfolio}\n")

        # STEP 4:
        # Compute the residual variance of the active portfolio:
        self.residual_variance_portfolio = sum((self.weights[ticker] ** 2) * self.residual_variance[ticker] for ticker in self.tickers)
#         print(f"Residual variance of the active portfolio: {self.residual_variance_portfolio}\n")

        # STEP 5:
        # Compute the initial position in the active portfolio:
        self.initial_position_portfolio = (self.alpha_portfolio * residual_variance_market) / (self.residual_variance_portfolio * risk_premium_market)
#         print(f"Initial position of the active portfolio: {self.initial_position_portfolio}\n")

        # STEP 6:
        # Compute the beta of the active portfolio:
        self.beta_portfolio = sum(self.weights[ticker] * self.beta[ticker] for ticker in self.tickers)
#         print(f"Beta of the active portfolio: {self.beta_portfolio}\n")

        # STEP 7:
        # Adjust the initial position in the active portfolio
        self.adjusted_position_portfolio = self.initial_position_portfolio / (1 + (1 - self.beta_portfolio) * self.initial_position_portfolio)
#         print(f"Adjusted position of the active portfolio: {self.adjusted_position_portfolio}")

        # STEP 8:
        # Optimal risky portfolio now has weights:
        self.final_weight_market = 1 - self.adjusted_position_portfolio
        self.weights = {ticker: weight * self.adjusted_position_portfolio for ticker, weight in self.weights.items()}

        print("------------------------------")
        print("Final Weights:")
        print("Weight Market S&P:", self.final_weight_market)
        print(f"Weights: {self.weights}")
        print("------------------------------")

        # STEP 9:
        # Calculate the risk premium of P (Optimal risky portfolio):
        self.risk_premium_portfolio = (self.final_weight_market + self.adjusted_position_portfolio * self.beta_portfolio) * risk_premium_market + self.adjusted_position_portfolio * self.alpha_portfolio
        print("Risk premium of portfolio: ", self.risk_premium_portfolio)

        # STEP 10:
        # Compute the variance of Portfolio:
        self.portfolio_variance = (self.final_weight_market + self.adjusted_position_portfolio * self.beta_portfolio) * (self.final_weight_market + self.adjusted_position_portfolio * self.beta_portfolio) * residual_variance_market + self.adjusted_position_portfolio * self.adjusted_position_portfolio * self.residual_variance_portfolio
        print("Variance of portfolio: ", self.portfolio_variance)

        self.sharpe_ratio = self.risk_premium_portfolio / (self.portfolio_variance ** 0.5)
        print(f"Sharpe ratio: {self.sharpe_ratio}")




### Random Forest for predicting market index values

In [5]:
# Returns of stocks and market
returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
for ticker in tickers:
    returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
    
# Future marker returns prediction
market_x = [returns["MARKET"][i:i+10] for i in range(len(returns["MARKET"])-20)]
market_y = [returns["MARKET"][i+10] for i in range(10, len(returns["MARKET"])-10)]

market_test = [returns["MARKET"][i:i+10]
              for i in range(len(returns["MARKET"])-20, len(returns["MARKET"])-10)]

reg = RandomForestRegressor()
reg.fit(market_x, market_y)
market_future = reg.predict(market_test)
market_future

array([ 1.78002498e-03,  4.36629644e-04,  8.27775954e-04, -4.20436682e-04,
        8.51831463e-04, -4.85635086e-04, -1.18126292e-04, -9.21207259e-04,
       -4.63660953e-05,  8.54627324e-04])

In [6]:
rf_portfolio = PortfolioOptimization(tickers, returns, market_future, RandomForestRegressor())
rf_portfolio.fit_single_index_model()
rf_portfolio.treynor_black_optimization()


------------------------------
Final Weights:
Weight Market S&P: 0.47954334562627976
Weights: {'AAPL': 0.3573357364553992, 'AMZN': -0.10203202924791738, 'BRK-B': 0.07716448628194651, 'GOOGL': 0.14402914006837308, 'META': 0.008439606243815984, 'MSFT': 0.04329624350829634, 'NVDA': 0.08534865982351693, 'TSLA': 0.0011975966775863434, 'UNH': -0.05344247173663607, 'XOM': -0.040880313700660725}
------------------------------
Risk premium of portfolio:  0.01618942233441617
Variance of portfolio:  0.0008563501548866818
Sharpe ratio: 0.5532299233291206


### SVM

In [7]:
# Returns of stocks and market
returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
for ticker in tickers:
    returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
    
# Future marker returns prediction
market_x = [returns["MARKET"][i:i+10] for i in range(len(returns["MARKET"])-20)]
market_y = [returns["MARKET"][i+10] for i in range(10, len(returns["MARKET"])-10)]

market_test = [returns["MARKET"][i:i+10]
              for i in range(len(returns["MARKET"])-20, len(returns["MARKET"])-10)]

reg = SVR()
reg.fit(market_x, market_y)
    
market_future = reg.predict(market_test)
market_future

array([-0.01122951, -0.01081277, -0.01130069, -0.01112012, -0.01126922,
       -0.01098801, -0.01194405, -0.01175388, -0.01087088, -0.01175548])

In [8]:
svm_portfolio = PortfolioOptimization(tickers, returns, market_future, SVR())
svm_portfolio.fit_single_index_model()
svm_portfolio.treynor_black_optimization()


------------------------------
Final Weights:
Weight Market S&P: 1.2760004299542536
Weights: {'AAPL': -0.09460889772480476, 'AMZN': 0.004028195545082886, 'BRK-B': -0.1129930862822095, 'GOOGL': -0.04570023927430623, 'META': 0.0024355674427593726, 'MSFT': -0.0747398611365536, 'NVDA': 0.0052622322702609685, 'TSLA': -0.009641218857108119, 'UNH': 0.03159037005430083, 'XOM': 0.01836650800832463}
------------------------------
Risk premium of portfolio:  -0.016077491156492072
Variance of portfolio:  4.4037142329353976e-05
Sharpe ratio: -2.4227506287041973


### Neural Network

In [9]:
# Returns of stocks and market
returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
for ticker in tickers:
    returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()
    
# Future marker returns prediction
market_x = [returns["MARKET"][i:i+10] for i in range(len(returns["MARKET"])-20)]
market_y = [returns["MARKET"][i+10] for i in range(10, len(returns["MARKET"])-10)]
market_test = [returns["MARKET"][i:i+10]
              for i in range(len(returns["MARKET"])-20, len(returns["MARKET"])-10)]

reg = MLPRegressor(hidden_layer_sizes=(100,100))
reg.fit(market_x, market_y)
market_future = reg.predict(market_test)
market_future

array([0.00183817, 0.00027691, 0.0006559 , 0.00202799, 0.00094487,
       0.00223394, 0.00027415, 0.00286747, 0.00042036, 0.00108245])

In [10]:
nn_portfolio = PortfolioOptimization(tickers, returns, market_future, MLPRegressor(hidden_layer_sizes=(100,100)))
nn_portfolio.fit_single_index_model()
nn_portfolio.treynor_black_optimization()


------------------------------
Final Weights:
Weight Market S&P: 1.051494906530617
Weights: {'AAPL': -0.14202002240947748, 'AMZN': -0.3593979897316505, 'BRK-B': -0.13676511352586224, 'GOOGL': -0.4904384619130814, 'META': 0.9420563005363616, 'MSFT': -0.19173132031604043, 'NVDA': 0.320883396549968, 'TSLA': 0.0006030174029592947, 'UNH': -0.12844721722204633, 'XOM': 0.13376250409825235}
------------------------------
Risk premium of portfolio:  0.02800470953700804
Variance of portfolio:  0.0016902980545377448
Sharpe ratio: 0.6811604547468092


In [11]:
class SingleIndexModel:
    def __init__(self, tickers, market_index_data, portfolio_data, reg_model):
        self.tickers = tickers
        self.market_index_data = market_index_data
        self.portfolio_data = portfolio_data
        self.reg_model = reg_model
        self.returns = {}
        self.beta = {}
        self.alpha = {}
        self.residual_variance = {}

    def calculate_returns(self):
        # Returns of stocks and market
        self.returns = {"MARKET": ((self.market_index_data / self.market_index_data.shift(1)) - 1).dropna().tolist()}
        for ticker in self.tickers:
            self.returns[ticker] = ((self.portfolio_data[ticker] / self.portfolio_data[ticker].shift(1))-1).dropna().tolist()

    def fit_regression(self):
        # Forecast market
        market_x = [self.returns["MARKET"][i:i+10] for i in range(len(self.returns["MARKET"])-20)]
        market_y = [self.returns["MARKET"][i+10] for i in range(10, len(self.returns["MARKET"])-10)]
        market_test = [self.returns["MARKET"][i:i+10] 
                       for i in range(len(self.returns["MARKET"])-20, len(self.returns["MARKET"])-10)]

        reg = self.reg_model
        reg.fit(market_x, market_y)
        market_future = reg.predict(market_test)

        # Forecast each stock
        for ticker in self.tickers:
            stock_x = [self.returns[ticker][i:i+10] for i in range(len(self.returns[ticker])-20)]
            stock_y = [self.returns[ticker][i+10] for i in range(10, len(self.returns[ticker])-10)]
            stock_test = [self.returns[ticker][i:i+10]
                          for i in range(len(self.returns[ticker])-20, len(self.returns[ticker])-10)]

            reg = self.reg_model
            reg.fit(stock_x, stock_y)
            stock_future = reg.predict(stock_test)

            single_index_reg = LinearRegression()
            single_index_reg.fit(np.array(market_future).reshape(-1, 1), y=stock_future)

            self.beta[ticker] = single_index_reg.coef_[0]
            self.alpha[ticker] = single_index_reg.intercept_

            y_pred = single_index_reg.predict(np.array(self.returns["MARKET"]).reshape(-1, 1))
            residuals = self.returns[ticker] - y_pred
            self.residual_variance[ticker] = np.var(residuals)

    def optimize_portfolio(self):
        # Compute the initial position of each security
        weights = {ticker: self.alpha[ticker] / self.residual_variance[ticker] for ticker in self.tickers}
        total_weight = sum(weights.values())
        weights = {ticker: weight / total_weight for ticker, weight in weights.items()}

        # Compute the alpha of the active portfolio
        alpha_portfolio = sum(weights[ticker] * self.alpha[ticker] for ticker in self.tickers)

        # Compute the residual variance of the active portfolio
        residual_variance_portfolio = sum((weights[ticker] ** 2) * self.residual_variance[ticker] for ticker in self.tickers)

        # Compute the initial position in the active portfolio
        residual_variance_market = 0.0114  # Variance of S&P 500
        risk_premium_market = 0.056
        initial_position_portfolio = (alpha_portfolio * residual_variance_market) / (residual_variance_portfolio * risk_premium_market)

        # Compute the beta of the active portfolio
        beta_portfolio = sum(weights[ticker] * self.beta[ticker] for ticker in self.tickers)

        # Adjust the initial position in the active portfolio
        adjusted_position_portfolio = initial_position_portfolio / (1 + (1 - beta_portfolio) * initial_position_portfolio)

        # Optimal risky portfolio now has weights
        final_weight_market = 1 - adjusted_position_portfolio
        weights = {ticker: weight * adjusted_position_portfolio for ticker, weight in weights.items()}

        # Calculate the risk premium of the portfolio
        risk_premium_portfolio = (final_weight_market + adjusted_position_portfolio * beta_portfolio) * risk_premium_market + adjusted_position_portfolio * alpha_portfolio

        # Compute the variance of the portfolio
        portfolio_variance = (final_weight_market + adjusted_position_portfolio * beta_portfolio) ** 2 * residual_variance_market + adjusted_position_portfolio ** 2 * residual_variance_portfolio

        # Calculate the Sharpe ratio
        sharpe_ratio = risk_premium_portfolio / (portfolio_variance ** 0.5)

        return weights, final_weight_market, sharpe_ratio


In [14]:
# reg_model = MLPRegressor(hidden_layer_sizes=(100,100))
portfolio = SingleIndexModel(tickers, market_index_data, portfolio_data, MLPRegressor(hidden_layer_sizes=(100,100)))
portfolio.calculate_returns()
portfolio.fit_regression()
weights, final_weight_market, sharpe_ratio = portfolio.optimize_portfolio()
print("------------------------------")
print("Final Weights:")
print("Weight Market S&P:", final_weight_market)
print(f"Weights: {weights}")
print("------------------------------")
print("Sharpe ratio:", sharpe_ratio)

------------------------------
Final Weights:
Weight Market S&P: 0.7155306944244944
Weights: {'AAPL': 0.34370695440319166, 'AMZN': -0.08097036566884633, 'BRK-B': 0.7697225837012904, 'GOOGL': -0.033434883335441756, 'META': 0.137473062609587, 'MSFT': 0.22904080570455324, 'NVDA': 0.2338238531865427, 'TSLA': 0.030072303561557254, 'UNH': -1.238867050778585, 'XOM': -0.10609795780834361}
------------------------------
Sharpe ratio: 0.657245717615602


### Capital asset pricing model (CAPM)

In [ ]:
from pypfopt import expected_returns
from pypfopt import plotting
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from pypfopt import expected_returns, risk_models, EfficientFrontier

# T-bill rate
risk_free_rate = 0.01

# apple_returns = (
#     (apple_data['Close'] / apple_data['Close'].shift(1))-1-risk_free_rate).dropna().tolist()
# google_returns = (
#     (google_data['Close'] / google_data['Close'].shift(1))-1-risk_free_rate).dropna().tolist()
# market_returns = (
#     (market_index_data['Close'] / market_index_data['Close'].shift(1))-1-risk_free_rate).dropna().tolist()

# Returns of stocks and market
returns = {"MARKET": ((market_index_data / market_index_data.shift(1)) - 1).dropna().tolist()}
for ticker in tickers:
    returns[ticker] = ((portfolio_data[ticker] / portfolio_data[ticker].shift(1))-1).dropna().tolist()


market_x = [returns["MARKET"][i:i+10] for i in range(len(returns["MARKET"])-20)]
market_y = [returns["MARKET"][i+10] for i in range(10, len(returns["MARKET"])-10)]
market_test = [returns["MARKET"][i:i+10]
              for i in range(len(returns["MARKET"])-20, len(returns["MARKET"])-10)]

reg = MLPRegressor(hidden_layer_sizes=(100,100))
reg.fit(market_x, market_y)
market_future = reg.predict(market_test)

# print(market_future)

future = {}
for ticker in tickers:
    
    single_index_reg = Ridge()
    single_index_reg.fit(np.array(returns["MARKET"]).reshape(-1, 1), y=returns[ticker])
    future[ticker] = single_index_reg.predict(np.array(market_future).reshape(-1, 1))

# single_index_reg = Ridge()
# single_index_reg.fit(np.array(market_returns).reshape(-1, 1), google_returns)
# google_future = single_index_reg.predict(
#     np.array(market_future).reshape(-1, 1))

future_returns = pd.DataFrame(future)
# print(future_returns)

S = risk_models.sample_cov(future_returns)
# S = risk_models.CovarianceShrinkage(future_returns).ledoit_wolf()
# plotting.plot_covariance(S, plot_correlation=True)
# You don't have to provide expected returns in this case


ef = EfficientFrontier(None, S)
ef.min_volatility()
weights = ef.clean_weights()
print(weights)
# weights = ef.max_sharpe(risk_free_rate=-0.2)
# cleaned_weights = ef.clean_weights()
# print(cleaned_weights)
# print(mu)

ret, volatility, sharpe_ratio = ef.portfolio_performance()
print("Expected annual return:", ret)
print("Annual volatility:", volatility)
print("Sharpe ratio:", sharpe_ratio)

### Arbitrage pricing theory and multifactor model

### Equity valuation model

### Black Litterman model 

### Algorithmic trading

In [1]:
import numpy as np
import pandas as pd
import talib
from sklearn.ensemble import RandomForestClassifier


apple_prices = apple_data['Close']

# Calculate Bollinger Bands for Apple
apple_bb_upper, apple_bb_middle, apple_bb_lower = talib.BBANDS(
    apple_prices, timeperiod=20)


# Calculate the Bollinger Bands percentages for Apple
apple_bb_percentage = (apple_prices - apple_bb_lower) / \
    (apple_bb_upper - apple_bb_lower)

# # Combine the Bollinger Bands percentages into a single DataFrame
# bb_percentages = pd.concat(
#     [apple_bb_percentage, google_bb_percentage], axis=1).dropna()

# Define the features and target variable for the machine learning model
# Independent variables (Bollinger Bands percentages for the past period)
X = apple_bb_percentage.values[:-1]
# Target variable (-1 for sell, 1 for buy)
y = np.where(apple_prices.values[1:] > apple_bb_upper[:-1], -1, 1)

# Train a machine learning model using random forest classifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X, y)

# Use the trained model to predict the trading signals for the testing set
# Ideally this will be your real time stock prices
y_pred = model.predict(X_test)

# Perform algorithmic trading based on the predicted signals (example logic)
capital = 100000  # Initial capital in USD
position = 0  # Current position (0 for neutral, 1 for long, -1 for short)

for i in range(len(y_pred)):
    if y_pred[i] == 1 and position != 1:  # Buy signal
        position = 1
        # Place a buy order based on your trading platform's API or logic

    elif y_pred[i] == -1 and position != -1:  # Sell signal
        position = -1
        # Place a sell order based on your trading platform's API or logic

    elif y_pred[i] == 0 and position != 0:  # Exit position
        position = 0
        # Close the existing position based on your trading platform's API or logic

# Calculate the final capital after the trading period
final_capital = capital  # Assume no transaction costs or slippage
# Calculate the final capital based on your trading platform's API or logic

print("Final Capital:", final_capital)


ModuleNotFoundError: No module named 'talib'